In [1]:
from shapely.geometry import Polygon, Point
import itertools
import geopandas as gpd
import pandas as pd
import rasterio as rio
from rasterio.features import rasterize
from rasterio import mask
from rasterio.plot import show
from rasterio.enums import Resampling
import matplotlib.pyplot as plt
import numpy as np
import shapely.wkt as wkt
from tqdm import tqdm
import os
import dask
import time
import gc
import re
from rioxarray.exceptions import NoDataInBounds
from rioxarray.merge import merge_arrays
from scipy.interpolate import NearestNDInterpolator

In [2]:
import xarray as xr
import rioxarray as riox
from xrspatial import convolution, focal, hillshade
from skimage.transform import resize
from dask.distributed import LocalCluster, Client
import distributed
import ctypes

In [3]:
dask.config.set({"distributed.nanny.environ.MALLOC_TRIM_THRESHOLD_": 0})
dask.config.set(scheduler='processes')

In [4]:
def trim_memory() -> int:
    libc = ctypes.CDLL("libc.so.6")
    return libc.malloc_trim(0)

os.environ["MALLOC_TRIM_THRESHOLD_"] = str(dask.config.get("distributed.nanny.environ.MALLOC_TRIM_THRESHOLD_"))

In [5]:
cluster_loc = 'hpc'

In [6]:
if cluster_loc == 'local':
    print('   setting up Local cluster...')
    cluster = LocalCluster(n_workers=8, threads_per_worker=2)
    client = Client(cluster)
    client.amm.start()
    display(client)
elif cluster_loc == 'hpc':
    from dask.distributed import LocalCluster, Client
    import dask_jobqueue as jq
    import dask
    from jupyter_server import serverapp
    # get the server address for porting
    try:
        jupServer = [x for x in serverapp.list_running_servers()][0]
    except IndexError:
        # manually copy/paste the server address
        jupServer = {'base_url': '/node/ceres19-compute-98-eth.scinet.local/17710/'}
    print('   setting up moderate cluster on HPC...')
    dask.config.set({'distributed.dashboard.link': jupServer['base_url'] + 'proxy/{port}/status'})
    partition='short',#'short','debug', 'mem', 'mem-low',
    num_processes = 4
    num_threads_per_processes = 2
    mem = 1.2*num_processes*num_threads_per_processes
    n_cores_per_job = num_processes*num_threads_per_processes
    clust = jq.SLURMCluster(queue=partition,
                            processes=num_processes,
                            cores=n_cores_per_job,
                            memory=str(mem)+'GB',
                            interface='ib0',
                            #interface='enp24s0f0',
                            local_directory='$TMPDIR',
                            death_timeout=30,
                            walltime='02:00:00',
                            job_extra_directives=["--output=/dev/null","--error=/dev/null"])
    client=Client(clust)
    #Scale Cluster 
    num_jobs=20
    clust.scale(jobs=num_jobs)
    try:
        client.wait_for_workers(n_workers=num_jobs*num_processes, timeout=60)
    except dask.distributed.TimeoutError as e:
        print(str(num_jobs*num_processes) + ' workers not available. Continuing with available workers.')
        #print(e)
        pass
    client.amm.start()
    display(client)

   setting up moderate cluster on HPC...


<Client: 'tcp://10.1.9.157:42151' processes=80 threads=160, memory=179.20 GiB>

In [7]:
# create function to normalize all data in range 0-1
def normalize_fn(image, image_suffix, stats_dict):
    if image_suffix in stats_dict.keys():
        min_tmp = stats_dict[image_suffix]['min']
        max_tmp = stats_dict[image_suffix]['max']
    else:
        # normalize to individual image if min/max stats not specified in dictionary
        min_tmp = np.min(image)
        max_tmp = np.max(image)
    return (image - min_tmp) / (max_tmp - min_tmp)

def calc_tpi(dtm, inner_r, outer_r, interpolate=True, values=True, bounds=(-3.0, 4.0)):
    cellsize_x, cellsize_y = convolution.calc_cellsize(dtm)
    kernel = convolution.annulus_kernel(cellsize_x, cellsize_y, outer_r, inner_r)
    tpi = dtm - focal.apply(dtm, kernel)
    tpi = tpi.rio.write_nodata(-9999.)
    tpi = tpi.where((tpi > bounds[0]) & (tpi < bounds[1]))
    if interpolate:
        if tpi.isnull().any().values:
            tpi = tpi.rio.interpolate_na(method='nearest')
    if values:
        return tpi.values
    else:
        return tpi

def calc_ndvi(ms, interpolate=True, values=True):
    ndvi = (ms.sel(band=4).astype('float32') - ms.sel(band=3).astype('float32'))\
            / (ms.sel(band=4).astype('float32') + ms.sel(band=3).astype('float32'))
    ndvi = ndvi.rio.write_nodata(-9999)
    ndvi = ndvi.where(ndvi != -9999)
    if interpolate:
        if ndvi.isnull().any().values:
            ndvi = ndvi.rio.interpolate_na(method='nearest')
    if values:
        return ndvi.values
    else:
        return ndvi

In [8]:
outDIR = '/90daydata/cper_neon_aop/cper_pdog_uas/mosaic_tiles/'
if not os.path.exists(outDIR):
    os.mkdir(outDIR)

In [9]:
res_fnl = 5

In [15]:
#past_subset = None
past_subset = ['29-30']

#baseDIR = '/mnt/d/'
baseDIR = '/90daydata/cper_neon_aop/cper_pdog_uas/'
#baseDIR = '/project/cper_neon_aop/cper_pdog_uas/uas_imagery/'

cper_f = './ground_data/pastures/cper_pdog_pastures_2017_clip.shp'

img_f_dict = {
    '5W': {
        'rgb': [os.path.join(baseDIR, '202109/outputs/202109_5W_RGB/CPER_202109_5W_RGB_ortho.tif')],
        'ms': [os.path.join(baseDIR, '202109/outputs/202109_5W_MS/CPER_202109_5W_MS_ortho.tif')],
        'dsm': [os.path.join(baseDIR, '202109/outputs/202109_5W_RGB/CPER_202109_5W_RGB_dsm.tif')]
    },
    '29-30': {
        'rgb': [os.path.join(baseDIR, '202109/outputs/202109_29_30_RGB/CPER_202109_29_30_RGB_ortho.tif')],
        'ms': [os.path.join(baseDIR, '202109/outputs/202109_29_30_MS/CPER_202109_29_30_North_MS_ortho.tif'),
              os.path.join(baseDIR, '202109/outputs/202109_29_30_MS/CPER_202109_29_30_South_MS_ortho.tif')],
        'dsm': [os.path.join(baseDIR, '202109/outputs/202109_29_30_RGB/CPER_202109_29_30_RGB_DSM.tif')]
    },
    '22W': {
        'rgb': [os.path.join(baseDIR, '202109/outputs/202109_22EW/CPER_202109_22EW_Flight1_RGB_ortho.tif'),
               os.path.join(baseDIR, '202109/outputs/202109_22EW/CPER_202109_22EW_Flight2_RGB_ortho.tif')],
        'ms': [os.path.join(baseDIR, '202109/outputs/202109_22EW/CPER_202109_22EW_Flight1_MS_ortho.tif'),
               os.path.join(baseDIR, '202109/outputs/202109_22EW/CPER_202109_22EW_Flight2_MS_ortho.tif')],
        'dsm': [os.path.join(baseDIR, '202109/outputs/202109_22EW/CPER_202109_22EW_Flight1_RGB_DSM.tif'),
                os.path.join(baseDIR, '202109/outputs/202109_22EW/CPER_202109_22EW_Flight2_RGB_DSM.tif')]
    },
    '22E': {
        'rgb': [os.path.join(baseDIR, '202109/outputs/202109_22EW/CPER_202109_22EW_Flight1_RGB_ortho.tif'),
               os.path.join(baseDIR, '202109/outputs/202109_22EW/CPER_202109_22EW_Flight2_RGB_ortho.tif'),
               os.path.join(baseDIR, '202109/outputs/202109_22EW/CPER_202109_22EW_Flight3_RGB_ortho.tif')],
        'ms': [os.path.join(baseDIR, '202109/outputs/202109_22EW/CPER_202109_22EW_Flight1_MS_ortho.tif'),
               os.path.join(baseDIR, '202109/outputs/202109_22EW/CPER_202109_22EW_Flight2_MS_ortho.tif')],
        'dsm': [os.path.join(baseDIR, '202109/outputs/202109_22EW/CPER_202109_22EW_Flight1_RGB_DSM.tif'),
               os.path.join(baseDIR, '202109/outputs/202109_22EW/CPER_202109_22EW_Flight2_RGB_DSM.tif'),
               os.path.join(baseDIR, '202109/outputs/202109_22EW/CPER_202109_22EW_Flight3_RGB_DSM.tif')]
    },
    'CN': {
        'rgb': [os.path.join(baseDIR, '202109/outputs/202109_CN_RGB/Orthos/CPER_CN_Flight2_202109_RGB_ortho.tif'),
               os.path.join(baseDIR, '202109/outputs/202109_CN_RGB/Orthos/CPER_CN_Flight3_202109_RGB_ortho.tif'),
               os.path.join(baseDIR, '202109/outputs/202109_CN_RGB/Orthos/CPER_CN_Flight4_202109_RGB_ortho.tif'),
               os.path.join(baseDIR, '202109/outputs/202109_CN_RGB/Orthos/CPER_CN_Flight5_202109_RGB_ortho.tif')],
        'ms': [os.path.join(baseDIR, '202109/outputs/202109_CN_MS/CPER_202109_CN_Flight2_MS_ortho.tif'),
              os.path.join(baseDIR, '202109/outputs/202109_CN_MS/CPER_202109_CN_Flight3_MS_ortho.tif'),
              os.path.join(baseDIR, '202109/outputs/202109_CN_MS/CPER_202109_CN_Flight4_MS_ortho.tif'),],
        'dsm': [os.path.join(baseDIR, '202109/outputs/202109_CN_RGB/DSMs/CPER_CN_Flight2_202109_RGB_DSM.tif'),
               os.path.join(baseDIR, '202109/outputs/202109_CN_RGB/DSMs/CPER_CN_Flight3_202109_RGB_DSM.tif'),
               os.path.join(baseDIR, '202109/outputs/202109_CN_RGB/DSMs/CPER_CN_Flight4_202109_RGB_DSM.tif'),
               os.path.join(baseDIR, '202109/outputs/202109_CN_RGB/DSMs/CPER_CN_Flight5_202109_RGB_DSM.tif')]
    }
}

if past_subset is not None:
    img_f_dict_tmp = img_f_dict.copy()
    img_f_dict = {}
    for k in img_f_dict_tmp:
         if k in past_subset:
                img_f_dict[k] = img_f_dict_tmp[k]

In [16]:
full_buff_size = 5
full_tile_size = 300
chunk_size = 500
inputs_fnl = ['rgb', 'dsm', 'tpi', 'ndvi']

In [17]:
if cluster_loc == 'hpc':
    import shutil
    # specifiy fast local storage directory
    user_dir = 'sean.kearney/'
    tmp_dir = os.path.join('/local/bgfs/' + user_dir, os.listdir('/local/bgfs/' + user_dir)[0])

In [ ]:
cper_gdf = gpd.read_file(cper_f)

for pasture in tqdm(img_f_dict):
    print('\n\n----------\nPasture: ' + pasture)
    if cluster_loc == 'hpc':
        print('Preparing data on fast local $TMPDIR storage')
        if not os.path.exists(os.path.join(tmp_dir, 'imagery')):
            os.mkdir(os.path.join(tmp_dir, 'imagery'))
        print('...copying RGB data')
        for idx, rgb_f in enumerate(tqdm(img_f_dict[pasture]['rgb'])):
            if not os.path.exists(os.path.join(tmp_dir, 'imagery', rgb_f.split('/')[-1])):
                shutil.copy(rgb_f, os.path.join(tmp_dir, 'imagery'))
            else:
                print('Skipping. File already copied.')
            img_f_dict[pasture]['rgb'][idx] = os.path.join(tmp_dir, 'imagery', rgb_f.split('/')[-1])
        print('...copying DSM data')
        for idx, dsm_f in enumerate(tqdm(img_f_dict[pasture]['dsm'])):
            if not os.path.exists(os.path.join(tmp_dir, 'imagery', dsm_f.split('/')[-1])):
                shutil.copy(dsm_f, os.path.join(tmp_dir, 'imagery'))
            else:
                print('Skipping. File already copied.')
            img_f_dict[pasture]['dsm'][idx] = os.path.join(tmp_dir, 'imagery', dsm_f.split('/')[-1])
        print('...copying Multispectral data')
        for idx, ms_f in enumerate(tqdm(img_f_dict[pasture]['ms'])):
            if not os.path.exists(os.path.join(tmp_dir, 'imagery', ms_f.split('/')[-1])):
                shutil.copy(ms_f, os.path.join(tmp_dir, 'imagery'))
            else:
                print('Skipping. File already copied.')
            img_f_dict[pasture]['ms'][idx] = os.path.join(tmp_dir, 'imagery', ms_f.split('/')[-1])
    
    print(img_f_dict[pasture])
    
    # make output directory
    if not os.path.exists(os.path.join(outDIR, pasture)):
        os.mkdir(os.path.join(outDIR, pasture))
    
    # get the bounding box of the pasture
    past_bbox = cper_gdf[cper_gdf['Past_Name_'] == pasture].buffer(
        full_buff_size).bounds.apply(lambda x: int(x))

    total_bounds = {'xmin': past_bbox['minx'],
                    'xmax': past_bbox['maxx'],
                    'ymin': past_bbox['miny'],
                    'ymax': past_bbox['maxy']}

    n_row_tiles = int(np.ceil((total_bounds['ymax'] - total_bounds['ymin'])/full_tile_size))
    n_col_tiles = int(np.ceil((total_bounds['xmax'] - total_bounds['xmin'])/full_tile_size))

    tile_ct = 0
    for full_r in range(n_row_tiles):
        print('running row: ' + str(full_r + 1) + ' of ' + str(n_row_tiles))
        for full_c in tqdm(range(n_col_tiles)):
            t0=time.time()

            ll_tile = [full_c * full_tile_size + total_bounds['xmin'],
                       full_r * full_tile_size + total_bounds['ymin']]
            ul_tile = [ll_tile[0], ll_tile[1] + full_tile_size]
            ur_tile = [x + full_tile_size for x in ll_tile]
            lr_tile = [ll_tile[0] + full_tile_size, ll_tile[1]]
            tile_poly = Polygon([ll_tile, ul_tile, ur_tile, lr_tile])

            ll = [full_c * full_tile_size + total_bounds['xmin'] - full_buff_size,
                  full_r * full_tile_size + total_bounds['ymin'] - full_buff_size]
            ul = [ll[0], ll[1] + full_tile_size + (full_buff_size * 2.0)]
            ur = [x + full_tile_size + (full_buff_size * 2.0) for x in ll]
            lr = [ll[0] + full_tile_size + (full_buff_size * 2.0), ll[1]]
            if not os.path.exists(os.path.join(outDIR, pasture, '_'.join(['ndvi', 
                                                                          str(round(ll_tile[0], 0)), 
                                                                          str(round(ll_tile[1], 0)), 
                                                                          str(res_fnl) + 'cm']) + '.nc')):

                newsize_r = int(round((ul[1] - ll[1]) / (res_fnl * 0.01), 0))
                newsize_c = int(round((lr[0] - ll[0]) / (res_fnl * 0.01), 0))
                if 'rgb' in inputs_fnl:
                    print('getting RGB')
                    t1=time.time()

                    rgb_xr_list = []
                    for rgb_f in img_f_dict[pasture]['rgb']:
                        with riox.open_rasterio(rgb_f, masked=True, cache=False) as rgb_src:
                            rgb_xr_list.append(rgb_src.sel(band=slice(0, 3),
                                                           x=slice(ll[0], lr[0]),
                                                           y=slice(ul[1], ll[1]), 
                                                           drop=True))
                    rgb_xr_list = [x for x in rgb_xr_list if not any([s == 0 for s in x.shape])]
                    rgb_xr_list = [x.where(x != 255, drop=True) for x in rgb_xr_list if not x.where(x != 255).isnull().all().values]
                    rgb_xr_list = [x.rio.write_nodata(-9999.).chunk({'band': -1,
                                                                   'x': chunk_size,
                                                                   'y': chunk_size}) for x in rgb_xr_list]

                    rgb_xr = merge_arrays(rgb_xr_list,
                                          bounds=(ll[0], ll[1], lr[0], ur[1]), 
                                          res=res_fnl*0.01, 
                                          crs=rio.CRS.from_epsg(32613),
                                          method='max',
                                          nodata=-9999)
                    rgb_xr = rgb_xr.where(rgb_xr != -9999)
                    if rgb_xr.isnull().any().values:
                        rgb_xr = rgb_xr.rio.interpolate_na(method='nearest')

                    rgb_xr = rgb_xr.rio.write_nodata(255).astype('uint8')

                    rgb_xr.to_netcdf(os.path.join(outDIR, pasture, '_'.join(['rgb', 
                                                                             str(round(ll_tile[0], 0)), 
                                                                             str(round(ll_tile[1], 0)), 
                                                                             str(res_fnl) + 'cm']) + '.nc'),
                                     format='NETCDF4', engine='h5netcdf')

                    #rgb_xr.close()
                    t2=time.time()
                    print('... completed in', round(t2 - t1, 1), 'secs')
                    del rgb_xr, rgb_xr_list
                if 'dsm' in inputs_fnl or 'tpi' in inputs_fnl:
                    t1 = time.time()
                    print('getting DSM')
                    dsm_xr_list = []
                    for dsm_f in img_f_dict[pasture]['dsm']:
                        with riox.open_rasterio(dsm_f, masked=True, cache=False) as dsm_src:
                            dsm_xr_list.append(dsm_src.sel(x=slice(ll[0], lr[0]),
                                                           y=slice(ul[1], ll[1]), 
                                                           drop=True))

                    dsm_xr_list = [x for x in dsm_xr_list if not any([s == 0 for s in x.shape])]
                    dsm_xr_list = [x.where(x > 0, drop=True).squeeze() for x in dsm_xr_list if not x.where(x > 0).isnull().all().values]
                    dsm_xr_list = [x.rio.write_nodata(-9999.).chunk({'x': chunk_size,
                                                                     'y': chunk_size}) for x in dsm_xr_list]

                    dsm_xr = merge_arrays(dsm_xr_list,
                                          bounds=(ll[0], ll[1], lr[0], ur[1]), 
                                          res=res_fnl*0.01, 
                                          crs=rio.CRS.from_epsg(32613),
                                          method='max',
                                          nodata=-9999)
                    dsm_xr = dsm_xr.where(dsm_xr > 0)

                    if dsm_xr.isnull().any().values:
                        dsm_xr = dsm_xr.rio.interpolate_na(method='nearest')

                    dsm_xr = dsm_xr.rio.write_nodata(-9999).astype('float32')

                    dsm_xr.to_netcdf(os.path.join(outDIR, pasture, '_'.join(['dsm', 
                                                                             str(round(ll_tile[0], 0)), 
                                                                             str(round(ll_tile[1], 0)), 
                                                                             str(res_fnl) + 'cm']) + '.nc'),
                                     format='NETCDF4', engine='h5netcdf')

                    t2=time.time()
                    print('... completed in', round(t2 - t1, 1), 'secs')
                if 'tpi' in inputs_fnl: 
                    t1 = time.time()
                    print('computing TPI')
                    # prepare an annulus kernel with a ring at a distance from 5-10 cells away from focal point
                    outer_radius = "0.75m"
                    inner_radius = "0.25m"
                    tpi_xr = calc_tpi(dsm_xr.chunk({'x': chunk_size,
                                                    'y': chunk_size}), 
                                                 inner_r=inner_radius, 
                                                 outer_r=outer_radius, 
                                                 interpolate=True,
                                                 values=False)
                    tpi_xr = tpi_xr.rio.write_nodata(-9999).astype('float32')

                    tpi_xr.to_netcdf(os.path.join(outDIR, pasture, '_'.join(['tpi', 
                                                                             str(round(ll_tile[0], 0)), 
                                                                             str(round(ll_tile[1], 0)), 
                                                                             str(res_fnl) + 'cm']) + '.nc'),
                                     format='NETCDF4', engine='h5netcdf')

                    t2=time.time()
                    print('... completed in', round(t2 - t1, 1), 'secs')
                    del dsm_xr, dsm_xr_list
                if 'ndvi' in inputs_fnl:
                    t1 = time.time()
                    print('computing NDVI')
                    ms_xr_list = []
                    for ms_f in img_f_dict[pasture]['ms']:
                        with riox.open_rasterio(ms_f, masked=True, cache=False) as ms_src:
                            ms_xr_list.append(ms_src.sel(band=[4, 3],
                                                           x=slice(ll[0], lr[0]),
                                                           y=slice(ul[1], ll[1]), 
                                                           drop=True))

                    ms_xr_list = [x for x in ms_xr_list if not any([s == 0 for s in x.shape])]
                    ms_xr_list = [x.where(x != 65535, drop=True) for x in ms_xr_list if not x.where(x != 65535).isnull().all().values]
                    ms_xr_list = [x.rio.write_nodata(-9999.).chunk({'x': chunk_size,
                                                                    'y': chunk_size}) for x in ms_xr_list]

                    ms_xr = merge_arrays(ms_xr_list,
                                          bounds=(ll[0], ll[1], lr[0], ur[1]), 
                                          res=res_fnl*0.01, 
                                          crs=rio.CRS.from_epsg(32613),
                                          method='max',
                                          nodata=-9999)
                    ms_xr = ms_xr.where(ms_xr != -9999)
                    ndvi_xr = calc_ndvi(ms_xr, values=False)

                    ndvi_xr = ndvi_xr.rio.write_nodata(-9999).astype('float32')

                    ndvi_xr.to_netcdf(os.path.join(outDIR, pasture, '_'.join(['ndvi', 
                                                                             str(round(ll_tile[0], 0)), 
                                                                             str(round(ll_tile[1], 0)), 
                                                                             str(res_fnl) + 'cm']) + '.nc'),
                                     format='NETCDF4', engine='h5netcdf')

                    #ms_xr.close()
                    t2=time.time()
                    #print('... completed in', round(t2 - t1, 1), 'secs')
                    del ms_xr, ms_xr_list, ndvi_xr

                    gc.collect()
                    client.run(gc.collect)
                    client.run(trim_memory)
                else:
                    print('Skipped! Tile mosaic already exists on disk.')


  0%|          | 0/1 [00:00<?, ?it/s]



----------
Pasture: 29-30
Preparing data on fast local $TMPDIR storage
...copying RGB data



100%|██████████| 1/1 [07:28<00:00, 448.90s/it]


...copying DSM data



100%|██████████| 1/1 [00:51<00:00, 51.84s/it]


...

{'rgb': ['/local/bgfs/sean.kearney/8145481/imagery/CPER_202109_29_30_RGB_ortho.tif'], 'ms': ['/local/bgfs/sean.kearney/8145481/imagery/CPER_202109_29_30_North_MS_ortho.tif', '/local/bgfs/sean.kearney/8145481/imagery/CPER_202109_29_30_South_MS_ortho.tif'], 'dsm': ['/local/bgfs/sean.kearney/8145481/imagery/CPER_202109_29_30_RGB_DSM.tif']}
running row: 1 of 6



100%|██████████| 9/9 [00:00<00:00, 329.93it/s]


running row: 2 of 6



  0%|          | 0/9 [00:00<?, ?it/s]

getting RGB
